In [1]:
import os
os.chdir("../")

In [2]:
%pwd

'c:\\Users\\teck1\\OneDrive\\Documents\\GitHub\\ai'

In [3]:
import tensorflow as tf

In [4]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    params_image_size: list
    params_batch_size: int

In [7]:
from DetectAnalytics.constants import *
from DetectAnalytics.utils.common import read_yaml, create_directories, save_json

In [8]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):
            self.config = read_yaml(config_filepath)
            self.params = read_yaml(params_filepath)
            create_directories([self.config.artifacts_root])

    def get_validation_config(self) -> EvaluationConfig:
          eval_config = EvaluationConfig(
                path_of_model="artifacts/training/model.h5",
                training_data="artifacts/data_ingestion/fighting_video",
                all_params=self.params,
                params_image_size=self.params.IMAGE_SIZE,
                params_batch_size=self.params.BATCH_SIZE

          )
          return eval_config

In [11]:
from urllib.parse import urlparse
import numpy as np
from sklearn.model_selection import train_test_split

In [14]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator_sample(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

    def _valid_generator(self):

        features = np.load("features.npy")
        labels = np.load("labels.npy")
        video_files_paths = np.load("video_files_paths.npy")

        encoded_labels = tf.keras.utils.to_categorical(labels)

        self.features_train, self.features_test, self.labels_train, self.labels_test = train_test_split(
            features, 
            encoded_labels, 
            test_size = 0.1, 
            shuffle = True, 
            random_state = 42
        )

    
    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = self.model.evaluate(self.features_test, self.labels_test)

    
    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

In [15]:
try:
    config = ConfigurationManager()
    val_config = config.get_validation_config()
    evaluation = Evaluation(val_config)
    evaluation.evaluation()
    evaluation.save_score()

except Exception as e:
   raise e

[2023-12-18 04:07:42,844: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-12-18 04:07:42,848: INFO: common: yaml file: params.yaml loaded successfully]
[2023-12-18 04:07:42,850: INFO: common: created directory at: artifacts]
7/7 [==============================] - 3s 293ms/step - loss: 1.0401 - accuracy: 0.7250
[2023-12-18 04:07:54,076: INFO: common: json file saved at: scores.json]
